# A Frames tutorial

## Building Frames

### From a Record collection

In [33]:
:ext TypeOperators
:ext DataKinds
:ext OverloadedStrings

import Data.Vinyl (Rec(RNil), ElField(Field))
import Frames (Record, (:->), (&:), toFrame, frameRow)
import Data.Text (Text)

type People = ["name" :-> Text, "age" :-> Int]

collection :: [Record People]
collection = (\(a, b) -> a &: b &: RNil) <$> [("Joe", 22), ("Bob", 32), ("Rudolph", 44)]

In [34]:
frame = toFrame collection
frameRow frame 2

{name :-> "Rudolph", age :-> 44}

### From A Pipes Producer



In [66]:
import Pipes (Producer, yield)
import qualified Pipes as P
import qualified Pipes.Prelude as P
import Frames (inCoreAoS, Frame)

prod :: Monad m => Producer (Record People) m ()
prod = sequence_ $ (\(a, b) -> yield $ a &: b &: RNil) <$> [("Joe", 22), ("Bob", 32), ("Rudolph", 44)]

frame <- inCoreAoS prod

In [36]:
mapM_ (print . frameRow frame) [0..2]

{name :-> "Joe", age :-> 22}
{name :-> "Bob", age :-> 32}
{name :-> "Rudolph", age :-> 44}

In [37]:
:t inCoreAoS

inCoreAoS :: forall (m :: * -> *) (rs :: [(Symbol, *)]). (PrimMonad m, MonadIO m, MonadMask m, RecVec rs) => Producer (Record rs) (SafeT m) () -> m (FrameRec rs)

### From a CSV

In [45]:
import Frames.CSV (writeCSV)

writeCSV "./tmp.dat" frame

In [46]:
import Frames.CSV (ParserOptions(ParserOptions), QuotingMode (RFC4180Quoting), readTableOpt)

csvParser :: ParserOptions
csvParser = ParserOptions (Just ["name", "age"]) "," (RFC4180Quoting '"')

producer = readTableOpt csvParser "./tmp.dat"
frame <- inCoreAoS producer :: (IO (Frame (Record People)))

In [47]:
mapM_ (print . frameRow frame) [0..2]

{name :-> "Joe", age :-> 22}
{name :-> "Bob", age :-> 32}
{name :-> "Rudolph", age :-> 44}

## Frame Operations

### Finding an element

In [72]:
:ext TypeApplications
:ext FlexibleContexts

import Data.Vinyl (rlens, rfield)
import Control.Lens (view)

type NameField = "name" :-> Text

type AgeField = "age" :-> Int

agefield = (rlens @AgeField . rfield)

istween :: Record People -> Bool
istween rec = let age=view agefield rec in age > 19 && age < 30

In [63]:
:t P.find

P.find :: forall (m :: * -> *) a. Monad m => (a -> Bool) -> Producer a m () -> m (Maybe a)

In [68]:
:t P.find istween

P.find istween :: forall (m :: * -> *). Monad m => Producer (Record People) m () -> m (Maybe (Record People))

In [69]:
:t P.each frame

P.each frame :: forall (m :: * -> *) x' x. Functor m => Proxy x' x () (Record People) m ()

In [78]:
fr <- P.find istween (P.each frame)
:t fr

case fr of
    Nothing -> error "not found"
    Just a -> view agefield a

fr :: Maybe (Record People)

22